## Code to find out the analytics in the Udacity enrollment
### First read the enrollments file and store it in a dictionary format 

In [2]:
import unicodecsv

with open('enrollments.csv','rb') as f:
    reader = unicodecsv.DictReader(f)
    enrollments = list(reader)
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': u'2015-01-14',
 u'days_to_cancel': u'65',
 u'is_canceled': u'True',
 u'is_udacity': u'True',
 u'join_date': u'2014-11-10',
 u'status': u'canceled'}

### Now read the 